In [19]:
!curl -O https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/AI+Engineer/Project+7%C2%A0-+D%C3%A9tectez+les+Bad+Buzz+gr%C3%A2ce+au+Deep+Learning/sentiment140.zip
!unzip -o sentiment140.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.9M  100 80.9M    0     0  1680k      0  0:00:49  0:00:49 --:--:-- 2160k:54  0:00:35  0:00:19 2210k     0  0:00:49  0:00:47  0:00:02 2097k
Archive:  sentiment140.zip
  inflating: training.1600000.processed.noemoticon.csv  


In [2]:
!pip install -r requirements.txt

In [3]:
import pandas as pd
import os
import json
import re
import string
import demoji
from tqdm import tqdm

import tensorflow as tf
import numpy as np
from joblib import Parallel, delayed
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import sys
import sklearn
import gensim
import tqdm as tq


In [4]:
print("Python Version:", sys.version)
print("TensorFlow Version:", tf.__version__)
print("Scikit-Learn Versionb:", sklearn.__version__)
print("Pandas Version:", pd.__version__)
print("Demoji Version:", demoji.__version__)
print("tqdm Version:", tq.__version__)
print("gensim Version:", gensim.__version__)


Python Version: 3.10.12 (main, Jul  5 2023, 15:02:25) [Clang 14.0.6 ]
TensorFlow Version: 2.15.0
Scikit-Learn Versionb: 1.2.2
Pandas Version: 1.5.3
Demoji Version: 1.1.0
tqdm Version: 4.66.1
gensim Version: 4.3.2


In [5]:
np.random.seed(42)
tf.random.set_seed(42)
tf.keras.backend.set_floatx('float32')
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED'] = str(1)

In [6]:
if tf.test.is_gpu_available():
    print("GPU is enabled!")
else:
    print("GPU is not enabled.")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is enabled!


2024-02-06 18:07:41.433754: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-02-06 18:07:41.433771: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2024-02-06 18:07:41.433777: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.00 GB
2024-02-06 18:07:41.433811: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-06 18:07:41.433824: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
cols = ['sentiment', 'timestamp', 'date', "query", "username", "comment"]
df = pd.read_csv("./training.1600000.processed.noemoticon.csv", encoding='ISO-8859-1', header=None, names=cols)
# df = df.sample(n=100000, random_state=42)

In [8]:
print(df.shape)
df.head()

(1600000, 6)


,sentiment,timestamp,date,query,username,comment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [9]:
def clean_tweet(doc):
  # Lower the code
  doc = doc.lower().strip()
  #remove emoji
  text = demoji.replace(doc, '')
  #remove links
  text = re.sub(r'http\S+|www.\S+', '', text)
  # # Remove mentions
  text = re.sub(r'@\w+', '', text)
  # Remove hashtag symbol but keep the text
  text = re.sub(r'#(\w+)', r'\1', text)
  # Keep only alphanumeric characters and spaces
  text = re.sub(r'[^a-zA-Z\s]', '', text)
  # Remove multiple spaces (replace them with a single space)
  text = re.sub(r'\s+', ' ', text).strip()

  return text

def clean_df(dataframe):
  df = dataframe.copy()
  # Keep only comment and sentiment columns
  df = df[["comment","sentiment"]]

  # negative field 0 = 0
  # Map positive field 4 = 1
  df.loc[df['sentiment'] == 4, 'sentiment'] = 1

  # Clean the comment
  df['comment_clean'] = parallelize_on_rows(df['comment'], clean_tweet)

  # Count the number of words from comment & comment_cleam
  df['words_nb'] = parallelize_on_rows(df['comment'], lambda x: len(x.split()))
  df['words_nb_clean'] = parallelize_on_rows(df['comment_clean'], lambda x: len(x.split()))

  # Only keep the clean words
  df = df[df['words_nb_clean'] > 3]

  # Remove duplicate
  df.drop_duplicates(subset='comment',inplace=True)

  return df


def parallelize_on_rows(data, func):
    r = Parallel(n_jobs=-1)(delayed(func)(i) for i in tqdm(data, desc="Processing"))
    return r

In [10]:
df = clean_df(df)
print(df.shape)
df.head()

Processing: 100%|██████████| 1600000/1600000 [00:02<00:00, 560917.16it/s]


(1469994, 5)


,comment,sentiment,comment_clean,words_nb,words_nb_clean
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,awww thats a bummer you shoulda got david carr...,19,16
1,is upset that he can't update his Facebook by ...,0,is upset that he cant update his facebook by t...,21,21
2,@Kenichan I dived many times for the ball. Man...,0,i dived many times for the ball managed to sav...,18,16
3,my whole body feels itchy and like its on fire,0,my whole body feels itchy and like its on fire,10,10
4,"@nationwideclass no, it's not behaving at all....",0,no its not behaving at all im mad why am i her...,21,20


In [11]:
# Function to vectorize a comment based on mean of all word vectors in the comment
def comment_to_vec(comment, model):
    vec = np.zeros(model.vector_size)
    num_words = 0
    for word in comment:
        if word in model.wv:
            vec += model.wv[word]
            num_words += 1
    if num_words > 0:
        vec /= num_words
    return vec

comments = [row.split() for row in df['comment_clean']]
word2vec_model = Word2Vec(comments, vector_size=100, window=5, min_count=1, workers=4)

# Vectorize all comments
vectorized_comments = np.array([comment_to_vec(comment, word2vec_model) for comment in comments])
# Preparing the labels
labels = df['sentiment'].values

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(vectorized_comments, labels, test_size=0.2, random_state=42)

model = Sequential([
    Dense(32, activation='relu', input_dim=100),  # Reduced the layer sizes for simplicity
    Dense(1, activation='sigmoid')  # Keeping the output layer same for binary classification
])

model.compile(optimizer="adam", loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=256, validation_split=0.1)

2024-02-06 18:10:12.625966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-06 18:10:12.625984: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/20
   1/4135 [..............................] - ETA: 21:30 - loss: 0.7818 - accuracy: 0.5156

2024-02-06 18:10:13.227226: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


4135/4135 [==============================] - 24s 6ms/step - loss: 0.5433 - accuracy: 0.7257 - val_loss: 0.5394 - val_accuracy: 0.7277
Epoch 2/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5397 - accuracy: 0.7290 - val_loss: 0.5379 - val_accuracy: 0.7309
Epoch 3/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5404 - accuracy: 0.7287 - val_loss: 0.5384 - val_accuracy: 0.7290
Epoch 4/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5408 - accuracy: 0.7284 - val_loss: 0.5377 - val_accuracy: 0.7300
Epoch 5/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5414 - accuracy: 0.7277 - val_loss: 0.5383 - val_accuracy: 0.7286
Epoch 6/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5417 - accuracy: 0.7277 - val_loss: 0.5407 - val_accuracy: 0.7280
Epoch 7/20
4135/4135 [==============================] - 23s 6ms/step - loss: 0.5424 - accuracy: 0.7274 - val_loss: 0.5394 - val_accuracy: 0.7

In [15]:
with tf.device('/GPU:0'):
    y_pred = model.predict(X_test)  
    y_pred_rounded = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)
    accuracy = accuracy_score(y_test, y_pred_rounded)
    print(accuracy)

9188/9188 [==============================] - 10s 1ms/step
0.7298528226286484


In [16]:
with tf.device('/CPU:0'):
    y_pred = model.predict(X_test)  
    y_pred_rounded = (y_pred > 0.5).astype(int)  # Convert probabilities to binary predictions (0 or 1)
    accuracy = accuracy_score(y_test, y_pred_rounded)
    print(accuracy)

9188/9188 [==============================] - 10s 1ms/step
0.5485528862343069


In [17]:
def predict_sentiment(comment, model, vector_model):
  cleaned_comment = comment.split() # placeholder for actual preprocessing
  vec = comment_to_vec(cleaned_comment, vector_model).reshape(1, -1)
  prediction = model.predict(vec)

  return prediction

In [18]:
print("GPU")
with tf.device('/GPU:0'):
    print(predict_sentiment("I am so sad, this is very bad news, terrible!", model, word2vec_model))
    print(predict_sentiment("I am so happy, this is very good news, congrats!", model, word2vec_model))

print("\nCPU")
with tf.device('/CPU:0'):
  print(predict_sentiment("I am so sad, this is very bad news, terrible!", model, word2vec_model))
  print(predict_sentiment("I am so happy, this is very good news, congrats!", model, word2vec_model))

GPU
1/1 [==============================] - 0s 31ms/step
[[0.09909298]]
1/1 [==============================] - 0s 16ms/step
[[0.659521]]

CPU
1/1 [==============================] - 0s 14ms/step
[[7.2889725e-06]]
1/1 [==============================] - 0s 17ms/step
[[1.1337466e-35]]
